In [1]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [16]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [2]:
!ls -lah /root/.cache/huggingface/hub/

total 4.0K
drwxr-xr-x 8 root root 303 Oct 24 11:46 .
drwxr-xr-x 5 root root  61 Oct 24 15:26 ..
drwxr-xr-x 6 root root  65 Aug 21 09:16 models--bigscience--bloom-1b7
drwxr-xr-x 6 root root  65 Oct 24 11:18 models--bigscience--bloom-7b1
drwxr-xr-x 6 root root  65 Aug 30 22:52 models--FlagAlpha--Llama2-Chinese-7b-Chat
drwxr-xr-x 6 root root  65 Oct 24 11:46 models--meta-llama--Llama-2-7b-chat-hf
drwxr-xr-x 6 root root  65 Sep  5 00:14 models--shibing624--text2vec-base-chinese
drwxr-xr-x 2 root root   6 Aug 29 13:57 models--TheBloke--Llama-2-7B-GGML
-rw------- 1 root root   0 Oct 24 11:18 tmpxe5e_q0g
-rw-r--r-- 1 root root   1 Jul 26 16:10 version.txt


In [3]:
!pip show transformers

Name: transformers
Version: 4.35.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [4]:
!nvidia-smi -L

GPU 0: NVIDIA A40 (UUID: GPU-84d1dbbd-b6a7-03b8-feb8-3ecfdf6b7b02)


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

2023-10-24 16:49:26.250618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 16:49:27.074761: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import torch
torch.cuda.device_count()

1

In [7]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [9]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "bigscience/bloom-7b1"


tokenizer = AutoTokenizer.from_pretrained(
    # "bigscience/bloom-7b1"
    pretrained_model_name_or_path=model_name,
    local_files_only=True
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    # "bigscience/bloom-7b1", 
    model_name,
    local_files_only=True,
    load_in_8bit=True, 
    device_map='auto',
    torch_dtype=torch.float16,
    do_sample=True,
    
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
!nvidia-smi

Thu Oct 26 07:42:00 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   28C    P0              68W / 300W |  35311MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
import pandas as pd
data = pd.read_csv("./data/warreconn_data.csv")

In [15]:
data["prediction"] = data.apply(lambda row: row["problem"] + " ->: " + row["answer"], axis=1)

In [16]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [34]:
dataset

Dataset({
    features: ['problem', 'role', 'answer', 'prediction'],
    num_rows: 6
})

In [36]:
dataset = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [37]:
dataset

Dataset({
    features: ['problem', 'role', 'answer', 'prediction', 'input_ids', 'attention_mask'],
    num_rows: 6
})

In [38]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [39]:
print_trainable_parameters(model)

trainable params: 7864320 || all params: 7076880384 || trainable%: 0.11112693126452029


In [40]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [21]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [22]:
print_trainable_parameters(model)

trainable params: 0 || all params: 7069016064 || trainable%: 0.0


In [23]:
# https://huggingface.co/docs/peft/quicktour

In [27]:
peft_config = LoraConfig(
    r=16, #attention heads #8
    lora_alpha=32, #alpha scaling 
    # target_modules=["q_proj", "v_proj"], #if you know the 
    lora_dropout=0.05, ##0.1
    # bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

In [28]:
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 7,864,320 || all params: 7,076,880,384 || trainable%: 0.11112693126452029


In [29]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
lr = 2e-4# 1e-3
batch_size = 16
num_epochs = 10

training_args = TrainingArguments(
    output_dir="Llama-2-7b-chat-hf_peft_model",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    # per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    # weight_decay=0.01,
    # evaluation_strategy="epoch",
    # save_strategy="epoch",
    # load_best_model_at_end=True,
    # ~~
    warmup_steps=100,
    max_steps=200,
    logging_steps=5,
    fp16=True
)

In [30]:
data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [31]:
peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [41]:
trainer = Trainer(
    model=peft_model, 
    train_dataset=data,#data['train'],
    args=training_args,
    data_collator=data_collator
)

In [42]:
trainer.train()

KeyError: 4

In [36]:
from huggingface_hub import notebook_login

notebook_login()

In [37]:
peft_model_name = "my_awesome_peft_model"
peft_model.push_to_hub(peft_model_name)

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ysw96/my_awesome_peft_model/commit/ef8640f38180f0d9763893af3adbf2c292c387d4', commit_message='Upload model', commit_description='', oid='ef8640f38180f0d9763893af3adbf2c292c387d4', pr_url=None, pr_revision=None, pr_num=None)